In [1]:
import requests as r
import pandas as pd
import re,os
from datetime import datetime
import json

In [8]:
import csv,json

def deduplicate_list_of_set(l):
    seen = set()
    result = []
    keys = list(l[0].keys())
    for dic in l:
        item = []
        for key in keys:
            item.append(dic[key])
        item_tuple = tuple(item)
        if item_tuple in seen:
            continue
        result.append(dic)
        seen.add(item_tuple)
    return result
        

def get_metadata_list(game_name)->list:
    metadata_list = []
    for i in range(0,6):
        with open(f'video_list/list_{i}.json') as fin:
            js = json.load(fin)
            metadata_list += js['data']
            print(len(metadata_list)) 
    metadata_list = deduplicate_list_of_set(metadata_list) 
    return metadata_list
    
metadata_list = get_metadata_list('minecraft')


99
198
296
396
495
595


In [9]:
def handle_chat_log(metadata_list):
    import os
    valid_list = []
    with open('video_list/urls.txt', 'w') as f:
        for id, metadata in enumerate(metadata_list):
            url = metadata['url']
            f.write(url+'\n')
        # chat = ChatDownloader().get_chat(url) 
        # for message in chat:                        # iterate over messages
        #     chat.print_formatted(message)
        # with open(f"{id}.json", "w")as f:
        #     f.write(json.dumps(chat, ensure_ascii=False))
        

handle_chat_log(metadata_list)

In [12]:
with open('scrap_chat_log.sh', 'w')as f:
    with open('video_list/urls.txt') as fin:
        raw = fin.readlines()
    print(f"Total {len(raw)} urls.")
    for i, line in enumerate(raw):
        f.write(f'chat_downloader {line.strip()} --output chat_log/{i}.json\n')

Total 590 urls.


In [ ]:
def filter_urls():
    l = []
    for file in os.listdir('chat_log'):
        l.append(int(file.split('.')[0]))
    with open('video_list/urls.txt') as fin:
        raw = fin.readlines()
        print(f"Before filtering, we have total {len(raw)} urls.")
    filtered_l = []
    for i in l:
        filtered_l.append(raw[i])
    with open('video_list/urls.txt', 'w') as fout:
        for line in filtered_l:
            fout.write(line)
    print(f"Before filtering, we have total {len(filtered_l)} urls.")

In [14]:
from spacy_langdetect import LanguageDetector
import spacy

def is_english(sent):
    nlp = spacy.load('en')
    nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)
    detected = doc = nlp(sent)._.language['language']
    if detected == "en":
        return True
    else:
        return False

print(is_english("Hello world!"))
print(is_english("今天天气真厚"))

/Users/caohanwen/miniconda3/envs/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [13]:
import json,csv
from spacy_langdetect import LanguageDetector
import spacy

def is_english(sent):
    nlp = spacy.load('en')
    nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)
    detected = doc = nlp(sent)._.language['language']
    if detected == "en":
        return True
    else:
        return False


def write_to_csv(csv_name, metadata_list):
    from datetime import datetime,timezone,timedelta

    def get_time(start_time):
        start_time_obj = datetime.fromisoformat(start_time[:-1]).astimezone(timezone.utc)
        return str(start_time_obj) 

    def add_time_eps(start_time, time_in_seconds:int):
        start_time_obj = datetime.fromisoformat(start_time[:-1]).astimezone(timezone.utc)
        delta = timedelta(seconds = time_in_seconds)
        return str(delta + start_time_obj)

    # gift price
    price = {1: 4.99, 2: 9.99, 3: 24.99,}
    
    fieldnames = ['id', # identifier of the video 
        'stream_id',
        'user_id',
        'user_login',
        'user_name',
        'title',
        'description',
        'url',
        'thumbnail_url',
        'viewable',
        'view_count',
        'language',
        'type',
        'duration',
        'video_created_at',
        'comment_created_at',
        'muted_segments',
        'time_in_seconds',  # time of comment sent after video begins (in second)
        'comment',  # comment message
        'author_name', # comment user
        'author_id', # identifier of author
        'display_author_name', # The name of the author which is displayed to the viewer. This may be different to name.
        'author_type',
        'author_images',
        'author_badges',
        'author_subscribe_status',
        'author_gender',
        'author_is_banned',
        'author_is_bot',
        'author_is_non_coworker',
        'author_is_original_poster',
        'author_is_verified',
        'message_type', # type of message
        'emotes',
        # gift states
        'has_gift',
        'gift_type',
        'gift_tier',
        'gift_amount',
        'gift_sum',
        'gift_receiver',
        ]
    with open(csv_name, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames = fieldnames)   
        writer.writeheader() 
        filename_list = os.listdir('chat_log')
        for i, filename in enumerate(filename_list):
            # record information about last community-gift-giving deed
            last_sender = None
            with open(f'chat_log/{filename}') as fin:
                js = json.load(fin)
            for item in js:
                row = {'id': i, }
                try:
                    del metadata_list[i]['id']
                except KeyError:
                    pass
                try:
                    del metadata_list[i]['language']# all english!
                except KeyError:
                    pass
                try:
                    created_at = metadata_list[i]['created_at']
                    del metadata_list[i]['created_at'] 
                    published_at = metadata_list[i]['published_ats']
                    del metadata_list[i]['published_ats']
                    assert(created_at == published_at)
                    published_at_str = get_time(published_at) 
                except KeyError:
                    pass
                row['video_created_at'] = published_at_str
                row.update(metadata_list[i])
                try:
                    row['time_in_seconds'] = item['time_in_seconds']
                    row['comment_created_at'] = add_time_eps(published_at, item['time_in_seconds'])
                except KeyError:
                    pass
                try:
                    row['comment'] =  item['message']
                    give_individual_sub = 'gifted a Tier' 
                    give_community_sub = 'is gifting'
                    if give_community_sub in item['message']:
                        last_sender = item['author']['id']
                        msg = item['message'][(item['message'].find(give_community_sub) + len(give_community_sub)):] 
                        msg = msg.split()
                        row['has_gift'] = True
                        row['gift_tier'] = int(msg[2])
                        row['gift_amount'] = int(msg[0])
                        row['gift_receiver'] = msg[5] + " " + msg[6]
                        row['gift_sum'] = row['gift_amount'] * price[row['gift_tier']]
                        row['gift_type'] = "To community" 
                    elif give_individual_sub in item['message']:
                        if last_sender != item['author']['id']:
                            msg = item['message'][item['message'].find(give_individual_sub) + len(give_individual_sub):]
                            msg = msg.split()
                            row['has_gift'] = True
                            row['gift_tier'] = int(msg[0])
                            row['gift_amount'] = 1
                            row['gift_receiver'] = msg[3]
                            row['gift_sum'] = row['gift_amount'] * price[row['gift_tier']]
                            row['gift_type'] = "To specific user"
                        else:
                            row['has_gift'] = False 
                    else:
                        row['has_gift'] = False
                except KeyError:
                    pass
                try:
                    row['author_name'] = item['author']['name']
                except KeyError:
                    pass
                try:
                    row['author_id'] = item['author']['id']
                except KeyError:
                    pass
                try:
                    row['display_author_name'] = item['author']['display_name']
                except KeyError:
                    pass
                try:
                    row['author_type'] = item['author']['type']
                except KeyError:
                    pass
                try:
                    row['author_images'] = item['author']['images']
                except KeyError:
                    pass
                try:
                    row['author_badges'] = item['author']['badges']
                    row['author_subscribe_status'] =  item['author']['badges']['description']
                except KeyError:
                    pass
                try:
                    row['author_gender'] = item['author']['gender']
                except KeyError:
                    pass
                try:
                    row['author_is_banned'] = item['author']['is_banned']
                except:
                    pass
                try:
                    row['author_is_bot' ] = item['author']['is_bot']
                except KeyError:
                    pass
                try:
                    row['author_is_non_coworker'] =  item['author']['is_non_coworker']
                except KeyError:
                    pass
                try:
                    row['author_is_original_poster'] = item['author']['is_original_poster']
                except KeyError:
                    pass
                try:
                    row['author_is_verified'] =item['author']['is_verified']
                except KeyError:
                    pass
                try:
                    row['message_type'] = item['message_type']
                except KeyError:
                    pass
                try:
                    row['emotes'] = item['emotes']
                except KeyError:
                    pass
                writer.writerow(row)
            
write_to_csv("clean/metadata.csv", metadata_list)

IndentationError: expected an indented block after function definition on line 3 (445862045.py, line 5)

In [ ]:
from pymkv import MKVFile
def extract_frames(mkv_file):
     
        
